# LLM-Based Semantic Cluster Analysis

This notebook performs cluster analysis on semantic data using Azure AI Foundry APIs.

## Features
- Handles multi-line semantic data grouped by `sequence_uuid`
- Uses Azure AI embeddings for semantic understanding
- Automatically generates cluster titles using LLM
- Visualizes clusters and their characteristics

## Prerequisites
Before running this notebook, ensure you have:
1. Azure AI Foundry API credentials
2. Set environment variables:
   - `AZURE_AI_ENDPOINT`: Your Azure AI endpoint URL
   - `AZURE_AI_KEY`: Your Azure AI API key
   - `AZURE_AI_MODEL_NAME`: The model name for embeddings (e.g., 'text-embedding-ada-002')
   - `AZURE_AI_CHAT_MODEL`: The chat model for cluster naming (e.g., 'gpt-4')

In [ ]:
# Import required libraries
import os
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Azure AI
from azure.ai.inference import ChatCompletionsClient, EmbeddingsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# Clustering and ML
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from dotenv import load_dotenv
from tqdm import tqdm

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")

## Configuration

In [ ]:
# Azure AI Configuration
AZURE_AI_ENDPOINT = os.getenv('AZURE_AI_ENDPOINT', '')
AZURE_AI_KEY = os.getenv('AZURE_AI_KEY', '')
AZURE_AI_MODEL_NAME = os.getenv('AZURE_AI_MODEL_NAME', 'text-embedding-ada-002')
AZURE_AI_CHAT_MODEL = os.getenv('AZURE_AI_CHAT_MODEL', 'gpt-4')

# Clustering Configuration
NUM_CLUSTERS = 5  # Adjust based on your data
CLUSTERING_METHOD = 'kmeans'  # Options: 'kmeans', 'dbscan'

# Validate configuration
if not AZURE_AI_ENDPOINT or not AZURE_AI_KEY:
    print("⚠️  Warning: Azure AI credentials not found in environment variables.")
    print("Please set AZURE_AI_ENDPOINT and AZURE_AI_KEY before proceeding.")
else:
    print("✓ Configuration loaded successfully")

## Initialize Azure AI Clients

In [ ]:
def initialize_azure_clients():
    """
    Initialize Azure AI clients for embeddings and chat completions.
    
    Returns:
        Tuple[EmbeddingsClient, ChatCompletionsClient]: Initialized clients
    """
    try:
        credential = AzureKeyCredential(AZURE_AI_KEY)
        
        embeddings_client = EmbeddingsClient(
            endpoint=AZURE_AI_ENDPOINT,
            credential=credential
        )
        
        chat_client = ChatCompletionsClient(
            endpoint=AZURE_AI_ENDPOINT,
            credential=credential
        )
        
        print("✓ Azure AI clients initialized successfully")
        return embeddings_client, chat_client
    except Exception as e:
        print(f"❌ Error initializing Azure AI clients: {str(e)}")
        return None, None

# Initialize clients
if AZURE_AI_ENDPOINT and AZURE_AI_KEY:
    embeddings_client, chat_client = initialize_azure_clients()
else:
    embeddings_client, chat_client = None, None
    print("⚠️  Skipping client initialization due to missing credentials")

## Data Loading and Preprocessing

In [ ]:
def load_and_group_data(file_path: str) -> pd.DataFrame:
    """
    Load semantic data from CSV and group by sequence_uuid.
    
    Args:
        file_path: Path to the CSV file with columns: sequence_uuid, semantic_data
        
    Returns:
        DataFrame with grouped semantic data
    """
    # Load data
    df = pd.read_csv(file_path)
    print(f"✓ Loaded {len(df)} rows from {file_path}")
    
    # Validate required columns
    required_columns = ['sequence_uuid', 'semantic_data']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")
    
    # Group by sequence_uuid and combine semantic data
    grouped_df = df.groupby('sequence_uuid').agg({
        'semantic_data': lambda x: ' '.join(x.astype(str))
    }).reset_index()
    
    grouped_df.rename(columns={'semantic_data': 'combined_semantic_data'}, inplace=True)
    
    print(f"✓ Grouped into {len(grouped_df)} unique sequence_uuids")
    
    return grouped_df

# Load sample data
data_file = 'sample_data.csv'
if os.path.exists(data_file):
    df_grouped = load_and_group_data(data_file)
    print("\nSample grouped data:")
    print(df_grouped.head())
else:
    print(f"⚠️  Data file '{data_file}' not found. Please provide your data file.")
    df_grouped = None

## Generate Embeddings using Azure AI

In [ ]:
def get_embeddings(texts: List[str], client: EmbeddingsClient, batch_size: int = 10) -> np.ndarray:
    """
    Generate embeddings for a list of texts using Azure AI.
    
    Args:
        texts: List of text strings to embed
        client: Azure AI EmbeddingsClient
        batch_size: Number of texts to process in each batch
        
    Returns:
        NumPy array of embeddings
    """
    all_embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch = texts[i:i + batch_size]
        
        try:
            response = client.embed(
                input=batch,
                model=AZURE_AI_MODEL_NAME
            )
            
            batch_embeddings = [item.embedding for item in response.data]
            all_embeddings.extend(batch_embeddings)
            
        except Exception as e:
            print(f"❌ Error generating embeddings for batch {i}: {str(e)}")
            # Return zero embeddings for failed batch
            all_embeddings.extend([np.zeros(1536) for _ in batch])
    
    return np.array(all_embeddings)

# Generate embeddings
if df_grouped is not None and embeddings_client is not None:
    texts = df_grouped['combined_semantic_data'].tolist()
    embeddings = get_embeddings(texts, embeddings_client)
    print(f"\n✓ Generated embeddings with shape: {embeddings.shape}")
    
    # Add embeddings to dataframe
    df_grouped['embedding'] = list(embeddings)
else:
    print("⚠️  Skipping embedding generation")
    embeddings = None

## Perform Clustering

In [ ]:
def perform_clustering(embeddings: np.ndarray, method: str = 'kmeans', n_clusters: int = 5) -> Tuple[np.ndarray, object]:
    """
    Perform clustering on embeddings.
    
    Args:
        embeddings: NumPy array of embeddings
        method: Clustering method ('kmeans' or 'dbscan')
        n_clusters: Number of clusters (for kmeans)
        
    Returns:
        Tuple of (cluster labels, clustering model)
    """
    # Standardize embeddings
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)
    
    if method == 'kmeans':
        model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = model.fit_predict(embeddings_scaled)
        
    elif method == 'dbscan':
        model = DBSCAN(eps=0.5, min_samples=2)
        labels = model.fit_predict(embeddings_scaled)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        
    else:
        raise ValueError(f"Unknown clustering method: {method}")
    
    # Calculate silhouette score
    if len(set(labels)) > 1:
        score = silhouette_score(embeddings_scaled, labels)
        print(f"✓ Clustering completed with {len(set(labels))} clusters")
        print(f"  Silhouette Score: {score:.3f}")
    else:
        print(f"✓ Clustering completed with {len(set(labels))} cluster")
    
    return labels, model

# Perform clustering
if embeddings is not None:
    cluster_labels, clustering_model = perform_clustering(
        embeddings, 
        method=CLUSTERING_METHOD, 
        n_clusters=NUM_CLUSTERS
    )
    
    # Add cluster labels to dataframe
    df_grouped['cluster'] = cluster_labels
    
    # Display cluster distribution
    print("\nCluster distribution:")
    print(df_grouped['cluster'].value_counts().sort_index())
else:
    print("⚠️  Skipping clustering")
    cluster_labels = None

## Generate Cluster Titles using LLM

In [ ]:
def generate_cluster_title(cluster_texts: List[str], client: ChatCompletionsClient) -> str:
    """
    Generate a descriptive title for a cluster using LLM.
    
    Args:
        cluster_texts: List of texts in the cluster
        client: Azure AI ChatCompletionsClient
        
    Returns:
        Generated cluster title
    """
    # Prepare sample texts (limit to avoid token limits)
    sample_size = min(10, len(cluster_texts))
    sample_texts = cluster_texts[:sample_size]
    
    # Create prompt
    texts_formatted = "\n".join([f"{i+1}. {text}" for i, text in enumerate(sample_texts)])
    
    prompt = f"""Analyze the following semantic data entries and provide a concise, descriptive title (3-6 words) that captures the main theme or topic:

{texts_formatted}

Provide only the title, nothing else."""
    
    try:
        response = client.complete(
            messages=[
                SystemMessage(content="You are a helpful assistant that analyzes text data and provides concise, descriptive titles."),
                UserMessage(content=prompt)
            ],
            model=AZURE_AI_CHAT_MODEL,
            temperature=0.3,
            max_tokens=50
        )
        
        title = response.choices[0].message.content.strip()
        # Remove quotes if present
        title = title.strip('"\'')
        return title
        
    except Exception as e:
        print(f"❌ Error generating cluster title: {str(e)}")
        return f"Cluster (Error generating title)"

def generate_all_cluster_titles(df: pd.DataFrame, client: ChatCompletionsClient) -> Dict[int, str]:
    """
    Generate titles for all clusters.
    
    Args:
        df: DataFrame with cluster assignments
        client: Azure AI ChatCompletionsClient
        
    Returns:
        Dictionary mapping cluster ID to title
    """
    cluster_titles = {}
    
    for cluster_id in sorted(df['cluster'].unique()):
        cluster_data = df[df['cluster'] == cluster_id]
        cluster_texts = cluster_data['combined_semantic_data'].tolist()
        
        print(f"Generating title for Cluster {cluster_id} ({len(cluster_texts)} items)...")
        title = generate_cluster_title(cluster_texts, client)
        cluster_titles[cluster_id] = title
        print(f"  → {title}")
    
    return cluster_titles

# Generate cluster titles
if df_grouped is not None and cluster_labels is not None and chat_client is not None:
    print("\nGenerating cluster titles...")
    cluster_titles = generate_all_cluster_titles(df_grouped, chat_client)
    
    # Add titles to dataframe
    df_grouped['cluster_title'] = df_grouped['cluster'].map(cluster_titles)
    
    print("\n✓ Cluster titles generated successfully")
else:
    print("⚠️  Skipping cluster title generation")
    cluster_titles = None

## Visualize Clusters

In [ ]:
def visualize_clusters(embeddings: np.ndarray, labels: np.ndarray, titles: Dict[int, str] = None):
    """
    Visualize clusters using PCA dimensionality reduction.
    
    Args:
        embeddings: NumPy array of embeddings
        labels: Cluster labels
        titles: Dictionary mapping cluster ID to title
    """
    # Reduce dimensions to 2D using PCA
    pca = PCA(n_components=2, random_state=42)
    embeddings_2d = pca.fit_transform(embeddings)
    
    # Create plot
    plt.figure(figsize=(12, 8))
    
    # Get unique clusters
    unique_labels = sorted(set(labels))
    colors = sns.color_palette('husl', len(unique_labels))
    
    # Plot each cluster
    for i, cluster_id in enumerate(unique_labels):
        mask = labels == cluster_id
        label = titles[cluster_id] if titles else f"Cluster {cluster_id}"
        
        plt.scatter(
            embeddings_2d[mask, 0],
            embeddings_2d[mask, 1],
            c=[colors[i]],
            label=label,
            alpha=0.6,
            s=100,
            edgecolors='black',
            linewidth=0.5
        )
    
    plt.xlabel('First Principal Component', fontsize=12)
    plt.ylabel('Second Principal Component', fontsize=12)
    plt.title('Semantic Data Clusters (PCA Visualization)', fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    variance_explained = pca.explained_variance_ratio_
    print(f"\nPCA Variance Explained:")
    print(f"  PC1: {variance_explained[0]:.2%}")
    print(f"  PC2: {variance_explained[1]:.2%}")
    print(f"  Total: {sum(variance_explained):.2%}")
    
    plt.show()

# Visualize clusters
if embeddings is not None and cluster_labels is not None:
    visualize_clusters(embeddings, cluster_labels, cluster_titles)
else:
    print("⚠️  Skipping visualization")

## Display Cluster Summary

In [ ]:
def display_cluster_summary(df: pd.DataFrame):
    """
    Display a summary of all clusters with sample data.
    
    Args:
        df: DataFrame with cluster assignments and titles
    """
    print("=" * 80)
    print("CLUSTER ANALYSIS SUMMARY")
    print("=" * 80)
    
    for cluster_id in sorted(df['cluster'].unique()):
        cluster_data = df[df['cluster'] == cluster_id]
        title = cluster_data.iloc[0]['cluster_title'] if 'cluster_title' in cluster_data.columns else f"Cluster {cluster_id}"
        
        print(f"\n{'─' * 80}")
        print(f"CLUSTER {cluster_id}: {title}")
        print(f"{'─' * 80}")
        print(f"Size: {len(cluster_data)} items")
        print(f"\nSample entries:")
        
        # Show up to 3 sample entries
        for idx, (_, row) in enumerate(cluster_data.head(3).iterrows(), 1):
            text = row['combined_semantic_data']
            # Truncate long texts
            if len(text) > 150:
                text = text[:150] + "..."
            print(f"  {idx}. [{row['sequence_uuid']}] {text}")
        
        if len(cluster_data) > 3:
            print(f"  ... and {len(cluster_data) - 3} more items")
    
    print(f"\n{'=' * 80}")
    print(f"Total: {len(df)} items across {len(df['cluster'].unique())} clusters")
    print(f"{'=' * 80}")

# Display summary
if df_grouped is not None and cluster_labels is not None:
    display_cluster_summary(df_grouped)
else:
    print("⚠️  No cluster data to display")

## Export Results

In [ ]:
def export_results(df: pd.DataFrame, output_file: str = 'clustered_results.csv'):
    """
    Export clustering results to CSV.
    
    Args:
        df: DataFrame with cluster assignments
        output_file: Output file path
    """
    # Prepare export dataframe (exclude embedding column)
    export_df = df.copy()
    if 'embedding' in export_df.columns:
        export_df = export_df.drop(columns=['embedding'])
    
    # Save to CSV
    export_df.to_csv(output_file, index=False)
    print(f"✓ Results exported to {output_file}")
    
    return export_df

# Export results
if df_grouped is not None and cluster_labels is not None:
    results_df = export_results(df_grouped)
    print("\nExported columns:", results_df.columns.tolist())
else:
    print("⚠️  No results to export")

## Usage Instructions

### Step 1: Set up Environment Variables
Create a `.env` file in the same directory with:
```
AZURE_AI_ENDPOINT=https://your-endpoint.cognitiveservices.azure.com/
AZURE_AI_KEY=your-api-key-here
AZURE_AI_MODEL_NAME=text-embedding-ada-002
AZURE_AI_CHAT_MODEL=gpt-4
```

### Step 2: Prepare Your Data
Create a CSV file with the following structure:
- `sequence_uuid`: Unique identifier for grouping related semantic data
- `semantic_data`: The text content to be analyzed

Example:
```csv
sequence_uuid,semantic_data
seq_001,First line of semantic data
seq_001,Second line related to seq_001
seq_002,Different semantic data
```

### Step 3: Configure Clustering
In the Configuration cell, adjust:
- `NUM_CLUSTERS`: Number of clusters (for K-means)
- `CLUSTERING_METHOD`: 'kmeans' or 'dbscan'

### Step 4: Run the Notebook
Execute all cells in order. The notebook will:
1. Load and group your data by `sequence_uuid`
2. Generate embeddings using Azure AI
3. Perform clustering
4. Generate descriptive titles for each cluster
5. Visualize the results
6. Export the clustered data

### Customization
- **Different clustering algorithms**: Modify `CLUSTERING_METHOD` to 'dbscan' for density-based clustering
- **Custom number of clusters**: Adjust `NUM_CLUSTERS` based on your data
- **Batch size**: Modify `batch_size` in `get_embeddings()` if you hit rate limits
- **Visualization**: Customize colors and plot settings in `visualize_clusters()`